In [1]:
!git clone https://github.com/Jarney903/Neural_Network_Charity_Analysis

## try git add, commit, push  

Cloning into 'Neural_Network_Charity_Analysis'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 149 (delta 26), reused 134 (delta 14), pack-reused 0
Receiving objects: 100% (149/149), 10.70 MiB | 27.12 MiB/s, done.
Resolving deltas: 100% (26/26), done.


To compare and contrast the performance of an SVM versus deep learning model, we'll try to build a binary classifier using the same input data. This adapted real-world dataset (Links to an external site.) contains bank telemarketing metrics that can be used to predict whether or not a customer is likely to subscribe to a banking service after being targeted by telemarketing advertisements. From this dataset, we want to build a binary classifier using an SVM and deep learning model and compare the predictive accuracy of either model.

In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
tele_df = pd.read_csv('Neural_Network_Charity_Analysis/Module Content/Resources/bank_telemarketing.csv')
tele_df.head()

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


Unlike neural networks and deep learning models, **SVMs** can handle unprocessed and processed tabular data. Regardless, we'll preprocesses the dataset and use the preprocessed data for training both models—this ensures a fair comparison. For our first preprocessing workflow, let's encode our categorical variables using Scikit-Learn's OneHotEncoder class.

First, we must make sure that none of our categorical variables require bucketing. To check this, let's get the column names of categorical variables and check their number of unique values. Add and run the following code to the notebook:

In [14]:
# Generate our categorical variable list
tele_cat = tele_df.dtypes[tele_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

In [15]:
# Create a OneHotEncoder instance

enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(tele_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


Once we have our encoded categorical variables, we need to merge our encoded columns back into our original DataFrame (as well as remove the unencoded columns). To replace the unencoded categorical variables with the encoded variables, add and run the following code to the notebook:

In [16]:
# Merge one-hot encoded features and drop the originals
tele_df = tele_df.merge(encode_df,left_index=True, right_index=True)
tele_df = tele_df.drop(tele_cat,1)
tele_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


Now, we must split our data into the training and testing sets prior to standardization to not incorporate the testing values into the scale—testing values are only for evaluation. To perform the training/test split and standardize our numerical variables, add and run the following code in the notebook:

In [17]:
# Remove loan status target from features data
y = tele_df.Subscribed_yes.values
X = tele_df.drop(columns=["Subscribed_no","Subscribed_yes"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

After standardizing variables in both the training and testing data, our dataset is ready for both models.

First, we'll train and **evaluate our SVM**.

For our purposes, we'll use the SVM's linear kernel to try and create a linear boundary between the "Subscribed_yes" versus "Subscribed_no" groups. To create our SVM model and test the performance, add and run the following code:

In [18]:
# Create the SVM model
svm = SVC(kernel='linear')

# Train the model
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")


 SVM model accuracy: 0.873


Next, we need to compile and evaluate our **deep learning model**. Again, we'll use our typical binary classifier parameters:

Our first hidden layer will have an input_dim equal to the length of the scaled feature data X , 10 neuron units, and will use the relu activation function.
Our second hidden layer will have 5 neuron units and also will use the relu activation function.
The loss function should be binary_crossentropy, using the adam optimizer.

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Lastly, we need to **train and evaluate our deep learning model.** Because this dataset contains fewer features than other datasets we have used previously, we only need to train over a maximum of 50 epochs. Again, we must add and run the following code:

In [21]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
715/715 [==============================] - 3s 2ms/step - loss: 0.3888 - accuracy: 0.8731
Epoch 2/50
715/715 [==============================] - 2s 2ms/step - loss: 0.3719 - accuracy: 0.8735
Epoch 3/50
715/715 [==============================] - 2s 2ms/step - loss: 0.3701 - accuracy: 0.8735
Epoch 4/50
715/715 [==============================] - 3s 3ms/step - loss: 0.3687 - accuracy: 0.8733
Epoch 5/50
715/715 [==============================] - 2s 2ms/step - loss: 0.3680 - accuracy: 0.8730
Epoch 6/50
715/715 [==============================] - 2s 2ms/step - loss: 0.3672 - accuracy: 0.8733
Epoch 7/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3662 - accuracy: 0.8735
Epoch 8/50
715/715 [==============================] - 2s 2ms/step - loss: 0.3657 - accuracy: 0.8733
Epoch 9/50
715/715 [==============================] - 2s 2ms/step - loss: 0.3647 - accuracy: 0.8735
Epoch 10/50
715/715 [==============================] - 2s 3ms/step - loss: 0.3645 - accuracy: 0.8733